In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/laxmimerit/twitter-suicidal-intention-dataset/master/twitter-suicidal_data.csv")

In [ ]:
data.head(10)

,tweet,intention
0,my life is meaningless i just want to end my l...,1
1,muttering i wanna die to myself daily for a fe...,1
2,work slave i really feel like my only purpose ...,1
3,i did something on the 2 of october i overdose...,1
4,i feel like no one cares i just want to die ma...,1
5,i am great and wonderful i am worth it except ...,1
6,i ll be dead just you wait and see my last wor...,1
7,health anxiety prompting some bad thoughts in ...,1
8,everything is okay but nothing feels okay i ve...,1
9,ptsd and alcohol i had some extremely horrible...,1


In [ ]:
data['intention'].value_counts()

0    5121
1    3998
Name: intention, dtype: int64

In [ ]:
!pip install pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git

  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-pgx0a72j
  Running command git clone -q https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-pgx0a72j
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.0.3-cp36-none-any.whl size=4514 sha256=8e5ce5996f261d0de3df1327faffb5b0959f56a0f7e58e10658f0d5ee57622a2
  Stored in directory: /tmp/pip-ephem-wheel-cache-vsut3rgx/wheels/a8/18/22/90afa4bd43247fb9a75b710a4a3fcd94966c022ce9e3c7d0a6
Successfully built preprocess-kgptalkie


In [ ]:
import preprocess_kgptalkie as ps
import re

def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)# contraction to expansion
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_rt(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [ ]:
data['tweet'] = data['tweet']. apply(lambda x: get_clean(x)) 

# tfidf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
vectorizer = TfidfVectorizer(max_features = 20000, ngram_range=(1,3), analyzer='char')

X = data['tweet']
y = data['intention']

X = vectorizer.fit_transform(X)

In [ ]:
X.shape

(9119, 10440)

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X, y, random_state =42, test_size =0.20)

In [ ]:
classifier = LinearSVC()
classifier.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[947  72]
 [ 79 726]]
              precision    recall  f1-score   support

           0       0.92      0.93      0.93      1019
           1       0.91      0.90      0.91       805

    accuracy                           0.92      1824
   macro avg       0.92      0.92      0.92      1824
weighted avg       0.92      0.92      0.92      1824



In [ ]:
x = 'n one cares me i will die alone'

x = get_clean(x)
vec = vectorizer.transform([x])

classifier.predict(vec)

array([1])

# suicidal intended tweet it is

In [ ]:
x = 'i am so happy and thanks a lot for making it special'

x = get_clean(x)
vec = vectorizer.transform([x])

classifier.predict(vec)

array([0])

# not suicidal intended

In [ ]:
import pickle
pickle.dump(classifier, open('model', 'wb'))